In [3]:
datapath='../../features'
Flods=[0,1,2,3]
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time
import os
import sklearn
from sklearn.neural_network import MLPClassifier
print(os.getcwd())

D:\Course\CS235\CS235Project\scripts


In [21]:
for flod in Flods:
    trainfile=open('{0}/cv_{1}_4_{2}_4/cv_train_features_{1}_4_{2}_4.csv'.format(datapath,flod,flod+1))
    testfile=open('{0}/cv_{1}_4_{2}_4/cv_test_features_{1}_4_{2}_4.csv'.format(datapath,flod,flod+1))
    trainX=[]
    trainY=[]
    testX=[]
    testY=[]
    for line in trainfile:
        line=line.split(',')[1:]
        if line[0]=='safe_type':
            continue
        line=[float(k) for k in line]
        trainX.append(line[1:])
        trainY.append(line[0])
    trainX=np.array(trainX)
    trainY=np.array(trainY)[:,np.newaxis].astype(int)
    #trainY[trainY==0]=-1
    print(trainX.shape,trainY.shape)
    
    for line in testfile:
        line=line.split(',')[1:]
        if line[0]=='safe_type':
            continue
        line=[float(k) for k in line]
        testX.append(line[1:])
        testY.append(line[0])
    testX=np.array(testX)
    testY=np.array(testY)[:,np.newaxis].astype(int)
    #testY[testY==0]=-1
    print(testX.shape,testY.shape)
    
    
    clf = MLPClassifier(activation='relu',solver='sgd', alpha=1e-5,hidden_layer_sizes=(100,100), random_state=1,shuffle=True,warm_start=True)
    clf.fit(trainX,trainY[:,0])
    ans=clf.predict(testX)
    print(ans)
#     err=np.sum(testY[:,0]*ans<=0)/testY.shape[0]
#     print(err)

(18000, 55) (18000, 1)
(6000, 55) (6000, 1)
[0 0 0 ... 0 0 0]
(18000, 55) (18000, 1)
(6000, 55) (6000, 1)
[0 0 0 ... 0 0 0]
(18000, 55) (18000, 1)
(6000, 55) (6000, 1)
[0 0 0 ... 0 0 0]
(18000, 55) (18000, 1)
(6000, 55) (6000, 1)
[0 0 0 ... 0 0 0]
